# Jupyter Notebook Setup and Usage Instructions

## Prerequisites

- **Python Version**: Ensure Python 3.10.12 is installed on your system.
- **Docker**: Make sure Docker is installed and the Docker engine is running.

---

## Installation Steps

### 1. Stop PostgreSQL if Running Locally
If you have PostgreSQL already running on your device, stop it to avoid port conflicts:

```bash
sudo systemctl stop postgresql
```

### 2. Start Docker Engine
Ensure Docker is running. If not, start it using:

```bash
sudo systemctl start docker
```

### 3. Execute the Docker Compose File
Run the following command to start the PostgreSQL instance with `pgvector`:

```bash
docker-compose up -d
```

### 4. Access the database to activate pgvector

```bash
docker exec -it pgvector psql -U postgres -d dwh
```

### 5. Activate `pgvector` Extension
Once inside the PostgreSQL prompt, execute:

```sql
CREATE EXTENSION IF NOT EXISTS vector;
```

### 6. Exit postres database

```sql
exit
```

### 7. Create a Python Virtual Environment (Optional)
If you prefer an isolated Python environment, create and activate a virtual environment:

```bash
python3 -m venv venv
source venv/bin/activate  # On Windows, use venv\Scripts\activate
```

### 8. Install Python Requirements
Install the required Python packages using the `requirements.txt` file:

```bash
pip install -r requirements.txt
```

---

## Usage

### 1. Open Jupyter Notebook
Start Jupyter Notebook in your project directory:

```bash
jupyter notebook
```

### 2. Run Notebook Cells
Open the notebook in your browser and execute the cells one after another to complete the task.



In [ ]:
import psycopg2
import umap
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import time

csv_file_path = 'msd.csv'
df = pd.read_csv(csv_file_path, nrows=10000)

features = [ 'duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'artist_familiarity', 'artist_hotttnesss','song_hotttnesss' ] # title but thats no feature
X = df[features].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Computing t-SNE embedding...")
tsne = TSNE(
    n_components=2,         # Zielraum: 2D-Einbettung
    perplexity=50,          # Balanciert lokale und globale Struktur
    learning_rate=500,      # Anpassungsgeschwindigkeit
    max_iter=5000,          # Maximale Iterationen
    init='pca',             # PCA-Initialisierung zur besseren Konvergenz
    random_state=42,        # Für reproduzierbare Ergebnisse
    metric='euclidean',     # Abstandsmessung: euklidisch
    n_jobs=-1               # Parallele Berechnung
)
start_time = time.perf_counter()
X_tsne = tsne.fit_transform(X)
end_time = time.perf_counter()

execution_time = end_time - start_time
print(f"t-SNE execution time: {execution_time:.4f} seconds")

print("Computing UMAP embedding...")
umap_model = umap.UMAP(
    n_components=2,         # Zielraum: 2D-Einbettung
    n_neighbors=100,         # Lokale/globaler Struktur-Balance
    min_dist=0.05,          # Clusterkompaktheit
    metric='euclidean',     # Abstandsmessung: euklidisch
    random_state=42,        # Für reproduzierbare Ergebnisse
    n_jobs=1                # Berechnung auf einem Kern
)
start_time = time.perf_counter()
X_umap = umap_model.fit_transform(X)
end_time = time.perf_counter()

execution_time = end_time - start_time
print(f"UMAP execution time: {execution_time:.4f} seconds")


records = [
    (f"[{','.join(map(str, map(float, X_tsne[i])))}]", 
     f"[{','.join(map(str, map(float, X_umap[i])))}]", 
     f"[{','.join(map(str, map(float, X[i])))}]", 
     str(df['title'][i].encode('utf-8', errors='ignore').decode('utf-8')))
    for i in range(len(X_tsne))
]

try:
    conn = psycopg2.connect(
        dbname="dwh",
        user="postgres",
        password="password",
        host="127.0.0.1",
        port="5432"
    )
    print("Connected to 'postgres' database successfully.")
except Exception as e:
    print(f"An error occurred while connecting to 'dwh': {e}")
finally:
    if 'conn' in locals() and conn is not None:
        conn.close()


try:
    print("Connecting to PostgreSQL database...")
    conn = psycopg.connect(
         user="postgres",
        password="password",
        host="127.0.0.1",
        port="5432",
        options="-c client_encoding=UTF8"
    )
    print("Connecting to PostgreSQL database...")
    
    with conn.cursor() as cur:

        print("Dropping the existing table if it exists...")
        cur.execute("DROP TABLE IF EXISTS music_embeddings;")
        conn.commit()
        print("Table dropped successfully.")

        cur.execute("""
        CREATE TABLE IF NOT EXISTS music_embeddings (
            id SERIAL PRIMARY KEY,
            tsne_vector VECTOR(2),
            umap_vector VECTOR(2),
            data VECTOR(11),
            titel VARCHAR(300)
        );
        """)
        
        print("Inserting data into database...")
        for tsne_vec, umap_vec, df,titel in records:
            cur.execute(
                "INSERT INTO music_embeddings (tsne_vector, umap_vector, data, titel) VALUES (%s, %s, %s, %s)",
                (tsne_vec, umap_vec, df ,titel)
            )
        
        conn.commit()
        print("Data inserted successfully.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # if 'conn' in locals() and conn is not None:
        conn.close()
        print("Database connection closed.")

An error occurred while connecting to 'dwh': 'utf-8' codec can't decode byte 0xbb in position 73: invalid start byte
Connecting to PostgreSQL database...
Connecting to PostgreSQL database...
Dropping the existing table if it exists...
Table dropped successfully.
An error occurred: Typ »vector« existiert nicht
LINE 4:             tsne_vector VECTOR(2),
                                ^
Database connection closed.


In [48]:
import psycopg2
import numpy as np
import matplotlib.pyplot as plt


# Connect to PostgreSQL database and fetch data
print("Connecting to PostgreSQL database...")
conn = psycopg2.connect(
    dbname="dwh",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

with conn.cursor() as cur:
    print("Fetching data from database...")
    cur.execute("""
        SELECT tsne_vector, umap_vector, data
        FROM music_embeddings;
    """)
    results = cur.fetchall()

# Close the database connection
conn.close()

titles = []
tsne_vectors = []
umap_vectors = []
counter = 0
data = []
for row in results:
    tsne_vectors.append([float(x) for x in row[0].strip('[]').split(',')]) 
    umap_vectors.append([float(x) for x in row[1].strip('[]').split(',')])
    data.append([float(x) for x in row[2].strip('[]').split(',')])
    if counter == 10000:
        break
    counter+=1
    

# # Convert to numpy arrays for visualization
X_tsne = np.array(tsne_vectors)
X_umap = np.array(umap_vectors)
X_data = np.array(data)

highlight_index = 6
labels = [ 'duration', 'end_of_fade_in', 'key', 'loudness', 'mode', 'start_of_fade_out', 'tempo', 'time_signature', 'artist_familiarity', 'artist_hotttnesss','song_hotttnesss']

color = []
for d in X_data:
    color.append(d[highlight_index])



plt.figure(figsize=(12, 6))

# Plot for t-SNE
plt.subplot(1, 2, 1)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=color, cmap='coolwarm', alpha=0.7)
plt.colorbar(label=labels[highlight_index])
plt.title("t-SNE Visualization of Music Data")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")

# Plot for UMAP
plt.subplot(1, 2, 2)
plt.scatter(X_umap[:, 0], X_umap[:, 1], c=color, cmap='coolwarm', alpha=0.7)
plt.colorbar(label=labels[highlight_index])
plt.title("UMAP Visualization of Music Data")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")

plt.show()


Connecting to PostgreSQL database...


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbb in position 79: invalid start byte